In [110]:
import os
import pandas as pd
from fuzzywuzzy import fuzz
from pyxdameraulevenshtein import damerau_levenshtein_distance
import regex as re
import sys

In [111]:
def replace_return(string):
    clean_string = re.sub(r"\n", "", string)
    clean_string = re.sub(r"nan", "", clean_string)
    clean_string = re.sub(r" ", "", clean_string)
    return clean_string

def score_response_damerau(response):
    if response <= 1:
        return 1
    else:
        return 0
    
def score_response_leveinshtein(response):
    if response > 95:
        return 1
    else:
        return 0

In [119]:
path = "/Users/Richard/PycharmProjects/NeuroLab_3.10/PsychoPy/scoring/Memory/delayed recall/nature/KA99GS130624_SNORE_recall_online_2024-06-13_22h54.28.095.csv"
df = pd.read_csv(path)
df 

,key_resp.keys,key_resp.rt,key_resp.duration,participant,date,expName,psychopyVersion,OS,frameRate,Response.text,...,recall_loop.thisIndex,recall_loop.ran,Word1,WORD_length1,Emo_condition,length_group_emo,Word2,WORD_length2,Ima_condition,length_group_ima
0,return,3.356,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Fisch,...,32.0,1.0,Strauch,7.0,emo_mid,long_word,Rebhuhn,7.0,ima_high,long_word
2,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Spargel,...,8.0,1.0,Knoblauch,9.0,emo_mid,long_word,Spargel,7.0,ima_high,long_word
3,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Larve,...,5.0,1.0,Rohr,4.0,emo_mid,short_word,Larve,5.0,ima_high,short_word
4,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Faultier,...,7.0,1.0,Zwiebel,7.0,emo_mid,long_word,Faultier,8.0,ima_high,long_word
5,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Gruß,...,12.0,1.0,Kardinal,8.0,emo_mid,long_word,Sellerie,8.0,ima_high,long_word
6,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Stinktier,...,31.0,1.0,Eidechse,8.0,emo_mid,long_word,Stinktier,9.0,ima_high,long_word
7,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Ei,...,23.0,1.0,Gaul,4.0,emo_mid,short_word,Ei,2.0,ima_high,short_word
8,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Krähe,...,18.0,1.0,Busch,5.0,emo_mid,short_word,Krähe,5.0,ima_high,short_word
9,NaN,NaN,NaN,KA99GS130624,2024-06-13_22h54.28.095,SNORE_recall_online,2023.1.3,Win32,60.096154,Schuh,...,29.0,1.0,Säugetier,9.0,emo_mid,long_word,Fledermaus,10.0,ima_high,long_word


In [120]:
df_short = df[["Word1","Word2", "Response.text"]].copy()  # Create a copy of the DataFrame slice
# Ensure all values are treated as strings, replacing NaNs with empty strings
df_short["Word2"] = df_short["Word2"].astype(str).fillna("")
df_short = df_short[df_short["Word2"] != "nan"]
df_short["Response.text"] = df_short["Response.text"].astype(str).fillna("")
df_short["Response.text"] = df_short["Response.text"].apply(replace_return)

#calculate distance based on Leveinstein algorithm
df_short["distance_leveinshtein"] = df_short.apply(lambda row: fuzz.token_sort_ratio(row["Word2"], row["Response.text"]), axis=1)

#calculate distance based on Damerau Levenshtein Distance
df_short["distance_damerau"] = df_short.apply(lambda row: damerau_levenshtein_distance(row["Word2"], row["Response.text"]), axis=1)

#Calculate score of the participant
df_short["scoring_leveinshtein"] = df_short["distance_leveinshtein"].apply(score_response_leveinshtein)
result_leveinshtein = df_short["scoring_leveinshtein"].sum()

df_short["scoring_damerau"] = df_short["distance_damerau"].apply(score_response_damerau)
result_damerau = df_short["scoring_damerau"].sum()

print(f"Result leveinshtein: {result_leveinshtein}")
print(f"Result damerau: {result_damerau}")

df_short[["Word1", "Word2", "Response.text", "distance_leveinshtein","distance_damerau"]].head(50)

Result leveinshtein: 30
Result damerau: 30


,Word1,Word2,Response.text,distance_leveinshtein,distance_damerau
1,Strauch,Rebhuhn,Fisch,17,6
2,Knoblauch,Spargel,Spargel,100,0
3,Rohr,Larve,Larve,100,0
4,Zwiebel,Faultier,Faultier,100,0
5,Kardinal,Sellerie,Gruß,18,7
6,Eidechse,Stinktier,Stinktier,100,0
7,Gaul,Ei,Ei,100,0
8,Busch,Krähe,Krähe,100,0
9,Säugetier,Fledermaus,Schuh,13,9
10,Barsch,Tiger,Tiger,100,0


# Manually checking programm 

In [76]:
need_to_check = df_short[(df_short["distance"] < 100) & (df_short["distance"] > 80)]
need_to_check

,Word1,Word2,Response.text,distance,scoring
9,Knoblauch,Spargel,Spagel,92,0


In [86]:
need_to_check.iloc[0, need_to_check.columns.get_loc('Response.text')]

'Spagel'

In [101]:
need_to_check

,Word1,Word2,Response.text,distance,scoring
9,Knoblauch,Spargel,Spagel,92,0


Spargel and Spagel: 1
Buch and BUch: 1
Daune and Daunnen: 2


In [100]:
# integrate this into a loop
if need_to_check.shape[0] > 0:
    print(f"Some words are almost the same. Please check if they are.\n "
          f"Is {need_to_check.iloc[0,need_to_check.columns.get_loc('Word2')]} close enough to {need_to_check.iloc[0, need_to_check.columns.get_loc('Response.text')]}?\n"
          f"if yes, press the y key if no press the n key.")
else:
    print(f"Result: {result}")
    sys.exit()
    
    
response = input("Press y or n to continue: ")

if response == "y":
    # set value to 1 if "y" is pressed
    word2_value = need_to_check.iloc[0, need_to_check.columns.get_loc('Word2')]
    df_short.loc[df_short["Word2"] == word2_value, "scoring"] = 1
    print("\nThanks for rating this row.")
elif response== "n":
    print("\nAll done.")
else:
    print("\nPlease press y or n to continue.")
    

Some words are almost the same. Please check if they are.
 Is Spargel close enough to Spagel?
if yes, press the y key if no press the n key.

Thanks for rating this row.


In [95]:
# Use .loc[] to avoid SettingWithCopyWarning
word2_value = need_to_check.iloc[0, need_to_check.columns.get_loc('Word2')]
df_short.loc[df_short["Word2"] == word2_value, "scoring"] = 1

df_short

,Word1,Word2,Response.text,distance,scoring
1,Hafer,Gras,Gras,100,1
2,Säugetier,Fledermaus,Fledermaus,100,1
3,Kolibri,Thunfisch,Thunfisch,100,1
4,Krokodil,Seeigel,Seeigel,100,1
5,Sprosse,Brombeere,Brombeere,100,1
6,Spore,Bulle,,0,0
7,Wolf,Buche,Buche,100,1
8,Languste,Getreide,,0,0
9,Knoblauch,Spargel,Spagel,92,1
10,Lorbeer,Walnuss,Walnuss,100,1


## Comparing fuzzy wuzzy functions

In [54]:
string1 = "Daune"
string2 = "Daunen"

ratio = fuzz.ratio(string1, string2)
partial_ratio = fuzz.partial_ratio(string1, string2)
token_sort_ratio = fuzz.token_sort_ratio(string1, string2)

print(f"Ratio: {ratio}")
print(f"Partial Ratio: {partial_ratio}")
print(f"Token Sort Ratio: {token_sort_ratio}")

Ratio: 91
Partial Ratio: 100
Token Sort Ratio: 91


## This uses the Damerau Levenshtein Distance

In [ ]:
correct_words = ["Spargel", "Buch", "Daune"]
typos = ["Spagel", "BUch", "Daunen"]

for correct_word, typo in zip(correct_words, typos):
    distance = damerau_levenshtein_distance(correct_word, typo)
    print(f"{correct_word} and {typo}: {distance}")